In [1]:
from run import prep_data
from nn import train_and_predict
import pandas as pd

In [2]:
X, y, X_test, labels = prep_data()

previous:  (931453, 24)
next:  (929615, 24)
transform:  (1861068, 254)
delta:  (929615, 254)
scale:  (929615, 254)
X_test:  (929615, 254)
threshold:  (1364729, 24) (1364729, 24)
transform:  (1364729, 105)
delta:  (1202476, 105) (1202476, 24)
paired_columns:  (1202476, 112)
scale:  (1202476, 112)


In [4]:
# get probability distribution for test
preds = train_and_predict(X, y, X_test)

Epoch: 0001, Cost: 4.59906244, Time: 111.30
Epoch: 0002, Cost: 4.48831224, Time: 114.55
Optimization Finished! Time to complete: 225.89


In [5]:
# construct results data frame
results = pd.DataFrame(preds.indices)

# change from int back to label
fn = lambda x: labels[int(x)]
results = results.applymap(fn)

results.head()

,0,1,2,3,4,5,6
0,ind_cco_fin_ult1,ind_recibo_ult1,ind_cno_fin_ult1,ind_nomina_ult1,ind_ctma_fin_ult1,ind_nom_pens_ult1,ind_ecue_fin_ult1
1,ind_cco_fin_ult1,ind_recibo_ult1,ind_cno_fin_ult1,ind_nomina_ult1,ind_ctma_fin_ult1,ind_nom_pens_ult1,ind_ecue_fin_ult1
2,ind_cco_fin_ult1,ind_recibo_ult1,ind_cno_fin_ult1,ind_nomina_ult1,ind_ctma_fin_ult1,ind_nom_pens_ult1,ind_ecue_fin_ult1
3,ind_cco_fin_ult1,ind_recibo_ult1,ind_cno_fin_ult1,ind_nomina_ult1,ind_ctma_fin_ult1,ind_nom_pens_ult1,ind_ecue_fin_ult1
4,ind_cco_fin_ult1,ind_recibo_ult1,ind_cno_fin_ult1,ind_nomina_ult1,ind_ctma_fin_ult1,ind_nom_pens_ult1,ind_ecue_fin_ult1


In [6]:
# join in a single columns
fn = lambda x:" ".join(x)
results['added_products'] = results.apply(fn, axis=1)

# add ncodpers
results['ncodpers'] = X_test.index

# delete other columns
results = results[['ncodpers', 'added_products']]

results.head()

,ncodpers,added_products
0,15889,ind_cco_fin_ult1 ind_recibo_ult1 ind_cno_fin_u...
1,15890,ind_cco_fin_ult1 ind_recibo_ult1 ind_cno_fin_u...
2,15892,ind_cco_fin_ult1 ind_recibo_ult1 ind_cno_fin_u...
3,15893,ind_cco_fin_ult1 ind_recibo_ult1 ind_cno_fin_u...
4,15894,ind_cco_fin_ult1 ind_recibo_ult1 ind_cno_fin_u...


In [7]:
# export to csv
print(results.shape)
results.to_csv('round10.csv', header=True, index=False)

(929615, 2)


In [13]:
# save classes to pickle
import pickle
pickle.dump( preds.indices, open( "preds.p", "wb" ) )

In [14]:
# load training data
from run import gen_data
X,y, labels = gen_data()

In [123]:
# isolate products
last_row = y[X['fecha_dato']=='2016-05-28']
last_row.index=X[X['fecha_dato']=='2016-05-28']['ncodpers']

In [124]:
last_row.head()

,ind_ahor_fin_ult1,ind_aval_fin_ult1,ind_cco_fin_ult1,ind_cder_fin_ult1,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_ctma_fin_ult1,ind_ctop_fin_ult1,ind_ctpp_fin_ult1,ind_deco_fin_ult1,...,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
ncodpers,,,,,,,,,,,,,,,,,,,,,
657640,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.0,0.0,0
657788,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.0,0.0,0
657795,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.0,0.0,0
657790,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,1,0,0,1.0,1.0,1
657794,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.0,0.0,0


In [126]:
def consolidate_products(arr):
    products_list = []
    for i, item in enumerate(arr):
        if item>0:
            products_list.append(i)
    return products_list

last_row['products'] = last_row.apply(consolidate_products, axis=1)

In [127]:
last_row['products'].head()

ncodpers
657640                         []
657788                        [2]
657795                        [2]
657790    [4, 12, 18, 21, 22, 23]
657794                    [2, 11]
Name: products, dtype: object

In [128]:
preds.indices

array([[ 2, 23,  4, ...,  6, 22, 12],
       [ 2, 23,  4, ...,  6, 22, 12],
       [ 2, 23,  4, ...,  6, 22, 12],
       ..., 
       [ 2, 23,  4, ...,  6, 22, 12],
       [ 2, 23,  4, ...,  6, 22, 12],
       [ 2, 23,  4, ...,  6, 22, 12]], dtype=int32)

In [129]:
# construct results data frame
results = pd.DataFrame(preds.indices, index=X_test.index)

In [131]:
# consolidate
def concat_products(row):
    return tuple([int(i) for i in row])

results['new_products'] = results.apply(concat_products, axis=1, reduce=True, raw=True)
results['new_products'].head()

ncodpers
15889    (2, 23, 4, 21, 6, 22, 12)
15890    (2, 23, 4, 21, 6, 22, 12)
15892    (2, 23, 4, 21, 6, 22, 12)
15893    (2, 23, 4, 21, 6, 22, 12)
15894    (2, 23, 4, 21, 6, 22, 12)
Name: new_products, dtype: object

In [132]:
# concatenate two dataframes
diff = pd.DataFrame(index=X_test.index)
diff['products'] = last_row['products']
diff['new_products'] = results['new_products']

In [133]:
diff.head()

,products,new_products
ncodpers,,
15889,"[2, 8, 18, 19]","(2, 23, 4, 21, 6, 22, 12)"
15890,"[4, 8, 12, 15, 18, 21, 22, 23]","(2, 23, 4, 21, 6, 22, 12)"
15892,"[2, 11, 12, 17, 18, 19, 23]","(2, 23, 4, 21, 6, 22, 12)"
15893,[19],"(2, 23, 4, 21, 6, 22, 12)"
15894,"[2, 12, 17, 18, 19, 21, 22, 23]","(2, 23, 4, 21, 6, 22, 12)"


In [136]:
# get the difference
def diff_products(row):
    return [i for i in row[1] if i not in row[0]]

diff['added_products'] = diff.apply(diff_products, axis=1)

In [143]:
diff.head()

,products,new_products,added_products
ncodpers,,,
15889,"[2, 8, 18, 19]","(2, 23, 4, 21, 6, 22, 12)","[23, 4, 21, 6, 22, 12]"
15890,"[4, 8, 12, 15, 18, 21, 22, 23]","(2, 23, 4, 21, 6, 22, 12)","[2, 6]"
15892,"[2, 11, 12, 17, 18, 19, 23]","(2, 23, 4, 21, 6, 22, 12)","[4, 21, 6, 22]"
15893,[19],"(2, 23, 4, 21, 6, 22, 12)","[2, 23, 4, 21, 6, 22, 12]"
15894,"[2, 12, 17, 18, 19, 21, 22, 23]","(2, 23, 4, 21, 6, 22, 12)","[4, 6]"


In [148]:
# now change i for string and join
def convert_products(row):
    mod_row = map(lambda x: labels[x], row)
    return " ".join(mod_row)
    
diff['added_products'] = diff['added_products'].apply(convert_products)

In [151]:
# check results
diff.head()

,products,new_products,added_products
ncodpers,,,
15889,"[2, 8, 18, 19]","(2, 23, 4, 21, 6, 22, 12)",ind_recibo_ult1 ind_cno_fin_ult1 ind_nomina_ul...
15890,"[4, 8, 12, 15, 18, 21, 22, 23]","(2, 23, 4, 21, 6, 22, 12)",ind_cco_fin_ult1 ind_ctma_fin_ult1
15892,"[2, 11, 12, 17, 18, 19, 23]","(2, 23, 4, 21, 6, 22, 12)",ind_cno_fin_ult1 ind_nomina_ult1 ind_ctma_fin_...
15893,[19],"(2, 23, 4, 21, 6, 22, 12)",ind_cco_fin_ult1 ind_recibo_ult1 ind_cno_fin_u...
15894,"[2, 12, 17, 18, 19, 21, 22, 23]","(2, 23, 4, 21, 6, 22, 12)",ind_cno_fin_ult1 ind_ctma_fin_ult1


In [153]:
del diff['products']
del diff['new_products']

In [155]:
diff.head()

,added_products
ncodpers,
15889,ind_recibo_ult1 ind_cno_fin_ult1 ind_nomina_ul...
15890,ind_cco_fin_ult1 ind_ctma_fin_ult1
15892,ind_cno_fin_ult1 ind_nomina_ult1 ind_ctma_fin_...
15893,ind_cco_fin_ult1 ind_recibo_ult1 ind_cno_fin_u...
15894,ind_cno_fin_ult1 ind_ctma_fin_ult1


In [157]:
diff.to_csv('round10b.csv', header=True, index=True)

In [ ]:
# run on cloud
# full training data
# train a lot
# try to converge cost
# remove products they already have at that position before doing the count